# GFM data discovery and download

In this notebook, we demonstrate how to query the GFM STAC collection and
download the found results using the Python library "pystac_client" together 
with a time and area of interest. 

In the STAC items a link to the respective asset (=file) is saved. This link is
used to download the file to a specified folder on the local machine.

In [ ]:
# Some needed imports
import rich.table
from datetime import datetime
from pystac_client import Client
from rich.console import Console
from shapely.geometry import box

console = Console()

### Specify search query

In [ ]:
# STAC endpoint
api_url = "https://dev.stac.eodc.eu/api/v1"

# STAC collection ID
collection_id = "GFM"

# Time range
time_range = (datetime(2022, 6, 1), datetime(2022, 6, 15))

# Area of interest
aoi = box(67, 23, 70, 29)

eodc_catalog = Client.open(api_url)

search = eodc_catalog.search(
    max_items=1000,
    collections=collection_id,
    intersects=aoi,
    datetime=time_range
)

items_eodc = search.item_collection()
console.print(f"On EODC we found {len(items_eodc)} items for the given search query")

### Some information about the found STAC items

In [ ]:
table = rich.table.Table(title="Assets in STAC Item")
table.add_column("Asset Key", style="cyan", no_wrap=True)
table.add_column("Description")
table.add_column("No Data Value")
table.add_column("Spatial resolution")
for asset_key, asset in items_eodc[0].assets.items():
    table.add_row(
        asset_key, 
        asset.title, 
        str(asset.extra_fields["raster:bands"][0]["nodata"]), 
        str(asset.extra_fields["raster:bands"][0]["spatial_resolution"]))

console.print(table)

### Download data

In [ ]:
import os
import urllib

download_root_path = "./downloaded_data/"

for item in items_eodc[:10]:
    download_path = os.path.join(download_root_path, item.collection_id, item.id)
    
    os.makedirs(download_path, exist_ok=True)
    
    for name, asset in item.assets.items():
        fpath = os.path.join(download_path, os.path.basename(asset.href))
        print(f"Downlading {fpath}")
        urllib.request.urlretrieve(asset.href, fpath)

print("Download done!")